In [1]:
from pyspark.ml.feature import RFormula as RF
from pyspark.ml.classification import LogisticRegression as LR
from pyspark.sql import SparkSession
import pyspark.sql.functions as fun

# Logistic Regression

In [2]:
spark = SparkSession.builder.appName("myapp").getOrCreate()

21/07/21 16:51:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/07/21 16:51:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true") \
.load("zoo.csv")

dataFrame = df.withColumn("IsMammal",fun.expr("CASE WHEN Type = 0 THEN 0 ELSE 1 END"))

In [4]:
supervised = RF(formula="IsMammal ~ . - Type - AnimalName")
fittedRF = supervised.fit(dataFrame)
PDF = fittedRF.transform(dataFrame)

In [5]:
PDF.where(PDF["IsMammal"] == 1).toPandas()

21/07/21 16:52:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,AnimalName,Hair,Feathers,Eggs,Milk,Airborne,Aquatic,Predator,Toothed,Backbone,...,Venomous,Fins,Legs,Tail,Domestic,Catsize,Type,IsMammal,features,label
0,aardvark,1,0,0,1,0,0,1,1,1,...,0,0,4,0,0,1,1,1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, ...",1.0
1,antelope,1,0,0,1,0,0,0,1,1,...,0,0,4,1,0,1,1,1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...",1.0
2,bass,0,0,1,0,0,1,1,1,1,...,0,1,0,1,0,0,4,1,"(0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, ...",1.0
3,bear,1,0,0,1,0,0,1,1,1,...,0,0,4,0,0,1,1,1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, ...",1.0
4,boar,1,0,0,1,0,0,1,1,1,...,0,0,4,1,0,1,1,1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, ...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,wallaby,1,0,0,1,0,0,0,1,1,...,0,0,2,1,0,1,1,1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...",1.0
97,wasp,1,0,1,0,1,0,0,0,0,...,1,0,6,0,0,0,6,1,"(1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
98,wolf,1,0,0,1,0,0,1,1,1,...,0,0,4,1,0,1,1,1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, ...",1.0
99,worm,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,7,1,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0


In [6]:
train, test = PDF.randomSplit([0.7,0.3])

In [7]:
logReg = LR(labelCol="label",featuresCol="features")
FLR = logReg.fit(train)

21/07/21 16:52:12 WARN Instrumentation: [3f24f87c] All labels are the same value and fitIntercept=true, so the coefficients will be zeros. Training is not needed.


In [9]:
final = FLR.transform(test)
final.toPandasdas()

,AnimalName,Hair,Feathers,Eggs,Milk,Airborne,Aquatic,Predator,Toothed,Backbone,...,Tail,Domestic,Catsize,Type,IsMammal,features,label,rawPrediction,probability,prediction
0,antelope,1,0,0,1,0,0,0,1,1,...,1,0,1,1,1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...",1.0,"[-inf, inf]","[0.0, 1.0]",1.0
1,bass,0,0,1,0,0,1,1,1,1,...,1,0,0,4,1,"(0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,"[-inf, inf]","[0.0, 1.0]",1.0
2,boar,1,0,0,1,0,0,1,1,1,...,1,0,1,1,1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, ...",1.0,"[-inf, inf]","[0.0, 1.0]",1.0
3,cheetah,1,0,0,1,0,0,1,1,1,...,1,0,1,1,1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, ...",1.0,"[-inf, inf]","[0.0, 1.0]",1.0
4,chub,0,0,1,0,0,1,1,1,1,...,1,0,0,4,1,"(0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,"[-inf, inf]","[0.0, 1.0]",1.0
5,crayfish,0,0,1,0,0,1,1,0,0,...,0,0,0,7,1,"(0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",1.0,"[-inf, inf]","[0.0, 1.0]",1.0
6,dolphin,0,0,0,1,0,1,1,1,1,...,1,0,1,1,1,"(0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,"[-inf, inf]","[0.0, 1.0]",1.0
7,flamingo,0,1,1,0,1,0,0,0,1,...,1,0,1,2,1,"(0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...",1.0,"[-inf, inf]","[0.0, 1.0]",1.0
8,frog,0,0,1,0,0,1,1,1,1,...,0,0,0,5,1,"(0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,"[-inf, inf]","[0.0, 1.0]",1.0
9,fruitbat,1,0,0,1,1,0,0,1,1,...,1,0,0,1,1,"(1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, ...",1.0,"[-inf, inf]","[0.0, 1.0]",1.0
